In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.ensemble import VotingClassifier
from sklearn.externals import joblib
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import log_loss

In [2]:
df = pd.read_csv("new_with_date.csv")

In [3]:
new_df = df[['aaDate','NUMBER_OF_CONTRACTS']]
sum_df = new_df.groupby(['aaDate'],as_index=False).sum()
sum_df2 = pd.DataFrame(data=sum_df)



df_old = pd.read_csv("end_with_date.csv")
old_df = df_old[['aaDate','NUMBER_OF_CONTRACTS_ENDED']]
old_sum_df = old_df.groupby(['aaDate'],as_index=False).sum()
sum_df_old = pd.DataFrame(data=old_sum_df)


In [4]:
sum_df2 = sum_df2.sort_values(by='aaDate', axis=0)
sum_df_old = sum_df_old.sort_values(by='aaDate', axis=0)

In [5]:
df_t = pd.read_csv("Contacts_Pre_2017.csv")

In [6]:
df_n = pd.get_dummies(df_t,columns=['CONTACT.TYPE'])

In [7]:
df_new = df_n.merge(sum_df2, left_on='START.DATE', right_on='aaDate', how='inner')

In [8]:
df_new = df_new.merge(sum_df_old,left_on='START.DATE', right_on='aaDate', how='inner')

In [9]:
labels = ['START.DATE','NUMBER_OF_CONTRACTS','NUMBER_OF_CONTRACTS_ENDED','Contacts']

In [10]:
sum_df_n = df_n.groupby(['START.DATE'],as_index=False).sum()
sum_df_nee = pd.DataFrame(data=sum_df_n)

In [11]:
sum_df_nee = sum_df_nee.sort_values(by='START.DATE', axis=0)

In [12]:
df_new_n = sum_df_nee.merge(sum_df2, left_on='START.DATE', right_on='aaDate', how='inner')
df_new_n = df_new_n.merge(sum_df_old,left_on='START.DATE', right_on='aaDate', how='inner')

In [13]:
labels = ['START.DATE','NUMBER_OF_CONTRACTS','NUMBER_OF_CONTRACTS_ENDED','Contacts']

In [14]:
df_att = df_new_n[labels];

In [15]:
df_k = df_att.drop(df_att[['START.DATE']],axis=1)

In [16]:
ss = StandardScaler()

ss.fit(df_k)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [17]:
from sklearn.preprocessing import scale

dk = scale(df_k)

In [18]:

ss =  df_att[['START.DATE']]
dfin = {"Scaled_data": dk}


In [19]:
y = df_att.NUMBER_OF_CONTRACTS.values
X = dk[:,0:2]


In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=42 )

In [21]:
#steps_GBC = [('ss',StandardScaler()),('pca', PCA()),('GradientBoosterClassifier', GradientBoostingClassifier())]
GBC = GradientBoostingClassifier()
#GBC = Pipeline(steps_GBC)
GBC.fit(X_train,y_train)
pred_gbc = GBC.predict(X_test)
pred_gbc_proba = GBC.predict_proba(X_test)

score_gbc = GBC.score(X_test,y_test)
print score_gbc


0.985915492958


In [22]:
from sklearn.metrics import mean_squared_error
print mean_squared_error(y_test, pred_gbc)

16.0824949698


In [23]:
joblib.dump(GBC, 'GBC_good.pkl')

['GBC_good.pkl']